In [56]:
!pip install smart_open scipy pydot
!apt update
!apt install -y graphviz

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


In [1]:
%config IPCompleter.use_jedi = False

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


import tensorflow as tf
from tensorflow import keras
import os
import pickle
from datetime import datetime
import numpy as np
from smart_open import open
from scipy.sparse import csr_matrix
import pydot

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
i_char = keras.Input(shape=(240,737,), name="Character-Input")
x_char = i_char
for _ in range(2):
    x_char = keras.layers.Conv1D(96, 5, activation="relu",padding='same')(x_char)
    x_char = keras.layers.Dropout(0.5)(x_char)
    x_char = keras.layers.MaxPooling1D(5,padding="same")(x_char)
x_char = keras.layers.Conv1D(64, 5, activation="relu",padding='same')(x_char)
x_char = keras.layers.Dropout(0.5)(x_char)
x_char = keras.layers.GlobalMaxPooling1D()(x_char)
x_char = keras.layers.Dense(32)(x_char)

In [10]:
i_bert = keras.Input(shape=(768,), name="BERT-Input")
x_bert = i_bert
for _ in range(2):
    x_bert = keras.layers.Dropout(0.5)(x_bert)
    x_bert = keras.layers.Dense(128)(x_bert)
#WHERE IS DROPOUT

In [12]:
i_bertconv = keras.Input(shape=(128, 768,), name="BERTconv-Input")
x_bertconv = i_bertconv
for _ in range(1):
    x_bertconv = keras.layers.Conv1D(128, 5, activation="relu",padding='same')(x_bertconv)
    x_bertconv = keras.layers.Dropout(0.5)(x_bertconv)
    x_bertconv = keras.layers.MaxPooling1D(5,padding="same")(x_bertconv)
x_bertconv = keras.layers.Conv1D(96, 5, activation="relu",padding='same')(x_bertconv)
x_bertconv = keras.layers.Dropout(0.5)(x_bertconv)
x_bertconv = keras.layers.GlobalMaxPooling1D()(x_bertconv)
x_bertconv = keras.layers.Dense(64)(x_bertconv)

In [13]:
i_electra = keras.Input(shape=(128, 768,), name="ELECTRA-Input")
x_electra = i_electra
for _ in range(1):
    x_electra = keras.layers.Conv1D(128, 5, activation="relu",padding='same')(x_electra)
    x_electra = keras.layers.Dropout(0.5)(x_electra)
    x_electra = keras.layers.MaxPooling1D(5,padding="same")(x_electra)
x_electra = keras.layers.Conv1D(96, 5, activation="relu",padding='same')(x_electra)
x_electra = keras.layers.Dropout(0.5)(x_electra)
x_electra = keras.layers.GlobalMaxPooling1D()(x_electra)
x_electra = keras.layers.Dense(64)(x_electra)

In [28]:
i_lda = keras.Input(shape=(50,), name="LDA-Input")
x_lda = i_lda
for _ in range(2):
    x_lda = keras.layers.Dropout(0.5)(x_lda)
    x_lda = keras.layers.Dense(16)(x_lda)

In [29]:
x = keras.layers.Concatenate(axis=1)([x_char,x_bert, x_bertconv, x_electra,x_lda])
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(128)(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(64)(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(32)(x)
x = keras.layers.Dropout(0.5)(x)

In [30]:
xop_polarity = keras.layers.Dense(16)(x)
xop_polarity = keras.layers.Dense(3, activation="softmax", name="xop_polarity")(xop_polarity)
# POLARITY PRETRAINING
xo_polarity = keras.layers.Dense(16)(x)
xo_polarity = keras.layers.Dense(3, activation="softmax", name="xo_polarity")(xo_polarity)

In [31]:
xo_relevance = keras.layers.Dense(16)(x)
xo_relevance = keras.layers.Dense(2, activation="softmax", name="xo_relevance")(xo_relevance)

In [32]:
xo_k_spam = keras.layers.Dense(16)(x)
xo_k_spam = keras.layers.Dense(2, activation="softmax", name="xo_k_spam")(xo_k_spam)

In [33]:
xo_k_joke = keras.layers.Dense(16)(x)
xo_k_joke = keras.layers.Dense(2, activation="softmax", name="xo_k_joke")(xo_k_joke)

In [34]:
xo_k_political = keras.layers.Dense(16)(x)
xo_k_political = keras.layers.Dense(2, activation="softmax", name="xo_k_political")(xo_k_political)

In [35]:
xo_k_technology = keras.layers.Dense(16)(x)
xo_k_technology = keras.layers.Dense(2, activation="softmax", name="xo_k_technology")(xo_k_technology)

In [36]:
xo_k_problem = keras.layers.Dense(16)(x)
xo_k_problem = keras.layers.Dense(2, activation="softmax", name="xo_k_problem")(xo_k_problem)

In [37]:
xo_k_money = keras.layers.Dense(16)(x)
xo_k_money = keras.layers.Dense(2, activation="softmax", name="xo_k_money")(xo_k_money)

In [38]:
xo_k_music = keras.layers.Dense(16)(x)
xo_k_music = keras.layers.Dense(2, activation="softmax", name="xo_k_music")(xo_k_music)

In [39]:
xo_k_brand = keras.layers.Dense(16)(x)
xo_k_brand = keras.layers.Dense(2, activation="softmax", name="xo_k_brand")(xo_k_brand)

In [28]:
# XOP Pretraining

In [4]:
basename = "traindata_rain_polarity_external_pretraining_"

In [5]:
basename = "traindata_rain_polarity_external_pretraining_"
y = np.load(basename+"Y.npz")
y_polarity = y["polarity"]
del y
X_char = np.load(basename+"char.npz")['arr_0']
X_bert = np.load(basename+"bert.npz")['arr_0']
X_bertconv = np.load(basename+"bert_conv.npz")['arr_0']
X_electra = np.load(basename+"electra.npz")['arr_0']
X_lda = np.load(basename+"lda.npz")['arr_0']

In [40]:
model = keras.Model(inputs=[ i_char, i_bert, i_bertconv, i_electra, i_lda],  outputs=[xop_polarity])


In [41]:
optimizer = keras.optimizers.Adam()
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy'])
model.fit(x=[X_char,X_bert, X_bertconv, X_electra,X_lda], y=y_polarity, epochs=15, batch_size=256, callbacks=[tensorboard_callback], validation_split=0.05)

Epoch 1/15
176/176 [==============================] - 549s 3s/step - loss: 3.1195 - accuracy: 0.3898 - val_loss: 0.5994 - val_accuracy: 0.7875
Epoch 2/15
176/176 [==============================] - 218s 1s/step - loss: 0.8015 - accuracy: 0.6428 - val_loss: 0.5541 - val_accuracy: 0.8158
Epoch 3/15
176/176 [==============================] - 38s 216ms/step - loss: 0.6725 - accuracy: 0.7179 - val_loss: 0.5280 - val_accuracy: 0.8150
Epoch 4/15
176/176 [==============================] - 29s 164ms/step - loss: 0.6163 - accuracy: 0.7531 - val_loss: 0.5089 - val_accuracy: 0.8129
Epoch 5/15
176/176 [==============================] - 34s 195ms/step - loss: 0.5990 - accuracy: 0.7587 - val_loss: 0.5603 - val_accuracy: 0.7798
Epoch 6/15
176/176 [==============================] - 37s 211ms/step - loss: 0.5798 - accuracy: 0.7670 - val_loss: 0.5141 - val_accuracy: 0.8171
Epoch 7/15
176/176 [==============================] - 36s 202ms/step - loss: 0.5683 - accuracy: 0.7705 - val_loss: 0.5014 - val_accura

In [43]:
del X_char, X_bert, X_lda, X_electra, X_bertconv, y_polarity, model

In [33]:
# Setting Model:

In [44]:
model = keras.Model(inputs=[
    i_char, i_bert, i_bertconv, i_electra, i_lda],
        outputs=[xo_polarity,xo_relevance, xo_k_spam, xo_k_joke, xo_k_political, xo_k_technology,xo_k_problem, xo_k_money, xo_k_music,xo_k_brand])


In [36]:
# Do Pretraining

In [45]:
basename = "traindata_merged_all_tweets_"
y = np.load(basename+"Y.npz")
y_polarity = y["polarity"] # Dreiwertig, Sparse Categorical Crossentropy
y_relevance = y["relevance"] # Binary crossentropy from logits
y_k_spam = y["spam"]
y_k_joke = y["joke"]
y_k_political = y["political"]
y_k_technology = y["technology"]
y_k_problem = y["problem"]
y_k_money = y["money"]
y_k_music = y["music"]
y_k_brand = y["brand"]
del y
X_char = np.load(basename+"char.npz")['arr_0']
X_bert = np.load(basename+"bert.npz")['arr_0']
X_bertconv = np.load(basename+"bert_conv.npz")['arr_0']
X_electra = np.load(basename+"electra.npz")['arr_0']
X_lda = np.load(basename+"lda.npz")['arr_0']



In [46]:
optimizer = keras.optimizers.Adam(1e-5)

In [47]:
losses = [keras.losses.SparseCategoricalCrossentropy() for _ in range(10)]

In [48]:
loss_weights = [1 for _ in range(10)]
#loss_weights[1]=1
model.compile(loss=losses, optimizer=optimizer, metrics=['accuracy'],loss_weights=loss_weights)
model.fit(x=[X_char,X_bert, X_bertconv, X_electra,X_lda], y=[y_polarity,y_relevance, y_k_spam, y_k_joke, y_k_political, y_k_technology,y_k_problem, y_k_money, y_k_music, y_k_brand], epochs=2048, batch_size=256, callbacks=[tensorboard_callback], validation_split=0.1)

Epoch 1/2048
13/13 [==============================] - 70s 4s/step - loss: 54.5862 - xo_polarity_loss: 8.1560 - xo_relevance_loss: 3.9767 - xo_k_spam_loss: 5.7063 - xo_k_joke_loss: 5.1535 - xo_k_political_loss: 4.6988 - xo_k_technology_loss: 4.5903 - xo_k_problem_loss: 5.4505 - xo_k_money_loss: 4.9625 - xo_k_music_loss: 3.9386 - xo_k_brand_loss: 7.9530 - xo_polarity_accuracy: 0.2827 - xo_relevance_accuracy: 0.5573 - xo_k_spam_accuracy: 0.4381 - xo_k_joke_accuracy: 0.5729 - xo_k_political_accuracy: 0.4485 - xo_k_technology_accuracy: 0.5079 - xo_k_problem_accuracy: 0.6200 - xo_k_money_accuracy: 0.5326 - xo_k_music_accuracy: 0.6212 - xo_k_brand_accuracy: 0.4666 - val_loss: 14.0327 - val_xo_polarity_loss: 2.4942 - val_xo_relevance_loss: 1.8172 - val_xo_k_spam_loss: 1.2244 - val_xo_k_joke_loss: 1.0848 - val_xo_k_political_loss: 1.4412 - val_xo_k_technology_loss: 0.7523 - val_xo_k_problem_loss: 1.3539 - val_xo_k_money_loss: 0.5680 - val_xo_k_music_loss: 1.0238 - val_xo_k_brand_loss: 2.2730 - 

13/13 [==============================] - 2s 153ms/step - loss: 27.8799 - xo_polarity_loss: 3.9837 - xo_relevance_loss: 2.1285 - xo_k_spam_loss: 2.4965 - xo_k_joke_loss: 2.7894 - xo_k_political_loss: 2.2654 - xo_k_technology_loss: 2.2976 - xo_k_problem_loss: 3.2951 - xo_k_money_loss: 2.5170 - xo_k_music_loss: 2.3656 - xo_k_brand_loss: 3.7410 - xo_polarity_accuracy: 0.3330 - xo_relevance_accuracy: 0.5415 - xo_k_spam_accuracy: 0.5098 - xo_k_joke_accuracy: 0.6202 - xo_k_political_accuracy: 0.4675 - xo_k_technology_accuracy: 0.5005 - xo_k_problem_accuracy: 0.5741 - xo_k_money_accuracy: 0.5349 - xo_k_music_accuracy: 0.5819 - xo_k_brand_accuracy: 0.4864 - val_loss: 8.2585 - val_xo_polarity_loss: 1.2335 - val_xo_relevance_loss: 1.0325 - val_xo_k_spam_loss: 0.5858 - val_xo_k_joke_loss: 0.5949 - val_xo_k_political_loss: 0.8829 - val_xo_k_technology_loss: 0.6532 - val_xo_k_problem_loss: 1.0486 - val_xo_k_money_loss: 0.5771 - val_xo_k_music_loss: 0.7915 - val_xo_k_brand_loss: 0.8584 - val_xo_polar

13/13 [==============================] - 2s 150ms/step - loss: 17.6495 - xo_polarity_loss: 2.5495 - xo_relevance_loss: 1.4515 - xo_k_spam_loss: 1.4630 - xo_k_joke_loss: 1.7738 - xo_k_political_loss: 1.3836 - xo_k_technology_loss: 1.6252 - xo_k_problem_loss: 2.0002 - xo_k_money_loss: 1.5833 - xo_k_music_loss: 1.6755 - xo_k_brand_loss: 2.1438 - xo_polarity_accuracy: 0.3614 - xo_relevance_accuracy: 0.5434 - xo_k_spam_accuracy: 0.5915 - xo_k_joke_accuracy: 0.6312 - xo_k_political_accuracy: 0.5134 - xo_k_technology_accuracy: 0.5141 - xo_k_problem_accuracy: 0.5943 - xo_k_money_accuracy: 0.6032 - xo_k_music_accuracy: 0.5634 - xo_k_brand_accuracy: 0.5631 - val_loss: 5.9701 - val_xo_polarity_loss: 0.9328 - val_xo_relevance_loss: 0.8928 - val_xo_k_spam_loss: 0.4142 - val_xo_k_joke_loss: 0.2649 - val_xo_k_political_loss: 0.7341 - val_xo_k_technology_loss: 0.5945 - val_xo_k_problem_loss: 0.6093 - val_xo_k_money_loss: 0.4255 - val_xo_k_music_loss: 0.5719 - val_xo_k_brand_loss: 0.5302 - val_xo_polar

13/13 [==============================] - 2s 150ms/step - loss: 12.5722 - xo_polarity_loss: 1.9230 - xo_relevance_loss: 1.2016 - xo_k_spam_loss: 0.9839 - xo_k_joke_loss: 1.0739 - xo_k_political_loss: 1.1211 - xo_k_technology_loss: 1.2985 - xo_k_problem_loss: 1.2293 - xo_k_money_loss: 1.0869 - xo_k_music_loss: 1.1012 - xo_k_brand_loss: 1.5527 - xo_polarity_accuracy: 0.3755 - xo_relevance_accuracy: 0.5239 - xo_k_spam_accuracy: 0.6582 - xo_k_joke_accuracy: 0.7005 - xo_k_political_accuracy: 0.5334 - xo_k_technology_accuracy: 0.5180 - xo_k_problem_accuracy: 0.6263 - xo_k_money_accuracy: 0.6535 - xo_k_music_accuracy: 0.6287 - xo_k_brand_accuracy: 0.5730 - val_loss: 5.0005 - val_xo_polarity_loss: 0.8169 - val_xo_relevance_loss: 0.9240 - val_xo_k_spam_loss: 0.3445 - val_xo_k_joke_loss: 0.1007 - val_xo_k_political_loss: 0.7319 - val_xo_k_technology_loss: 0.5159 - val_xo_k_problem_loss: 0.3746 - val_xo_k_money_loss: 0.2964 - val_xo_k_music_loss: 0.4299 - val_xo_k_brand_loss: 0.4659 - val_xo_polar

13/13 [==============================] - 2s 148ms/step - loss: 10.1527 - xo_polarity_loss: 1.6570 - xo_relevance_loss: 1.1197 - xo_k_spam_loss: 0.7884 - xo_k_joke_loss: 0.6686 - xo_k_political_loss: 0.9397 - xo_k_technology_loss: 1.1039 - xo_k_problem_loss: 0.9169 - xo_k_money_loss: 0.7778 - xo_k_music_loss: 0.9321 - xo_k_brand_loss: 1.2486 - xo_polarity_accuracy: 0.4049 - xo_relevance_accuracy: 0.5536 - xo_k_spam_accuracy: 0.6931 - xo_k_joke_accuracy: 0.7764 - xo_k_political_accuracy: 0.5551 - xo_k_technology_accuracy: 0.5556 - xo_k_problem_accuracy: 0.6867 - xo_k_money_accuracy: 0.7081 - xo_k_music_accuracy: 0.6549 - xo_k_brand_accuracy: 0.6175 - val_loss: 4.6352 - val_xo_polarity_loss: 0.7598 - val_xo_relevance_loss: 0.9091 - val_xo_k_spam_loss: 0.3066 - val_xo_k_joke_loss: 0.0757 - val_xo_k_political_loss: 0.6919 - val_xo_k_technology_loss: 0.4829 - val_xo_k_problem_loss: 0.3139 - val_xo_k_money_loss: 0.2575 - val_xo_k_music_loss: 0.4003 - val_xo_k_brand_loss: 0.4374 - val_xo_polar

...

13/13 [==============================] - 2s 146ms/step - loss: 2.8566 - xo_polarity_loss: 0.6345 - xo_relevance_loss: 0.3295 - xo_k_spam_loss: 0.1880 - xo_k_joke_loss: 0.1130 - xo_k_political_loss: 0.2026 - xo_k_technology_loss: 0.4215 - xo_k_problem_loss: 0.2152 - xo_k_money_loss: 0.1635 - xo_k_music_loss: 0.1695 - xo_k_brand_loss: 0.4192 - xo_polarity_accuracy: 0.7457 - xo_relevance_accuracy: 0.8578 - xo_k_spam_accuracy: 0.9461 - xo_k_joke_accuracy: 0.9744 - xo_k_political_accuracy: 0.9192 - xo_k_technology_accuracy: 0.7980 - xo_k_problem_accuracy: 0.9261 - xo_k_money_accuracy: 0.9554 - xo_k_music_accuracy: 0.9410 - xo_k_brand_accuracy: 0.7804 - val_loss: 3.2339 - val_xo_polarity_loss: 0.5427 - val_xo_relevance_loss: 0.8764 - val_xo_k_spam_loss: 0.1418 - val_xo_k_joke_loss: 0.0275 - val_xo_k_political_loss: 0.2744 - val_xo_k_technology_loss: 0.4105 - val_xo_k_problem_loss: 0.1608 - val_xo_k_money_loss: 0.1008 - val_xo_k_music_loss: 0.2783 - val_xo_k_brand_loss: 0.4208 - val_xo_polari

13/13 [==============================] - 2s 150ms/step - loss: 2.8193 - xo_polarity_loss: 0.6270 - xo_relevance_loss: 0.3196 - xo_k_spam_loss: 0.1978 - xo_k_joke_loss: 0.1142 - xo_k_political_loss: 0.1897 - xo_k_technology_loss: 0.4283 - xo_k_problem_loss: 0.2155 - xo_k_money_loss: 0.1455 - xo_k_music_loss: 0.1606 - xo_k_brand_loss: 0.4212 - xo_polarity_accuracy: 0.7420 - xo_relevance_accuracy: 0.8618 - xo_k_spam_accuracy: 0.9447 - xo_k_joke_accuracy: 0.9728 - xo_k_political_accuracy: 0.9268 - xo_k_technology_accuracy: 0.7900 - xo_k_problem_accuracy: 0.9246 - xo_k_money_accuracy: 0.9605 - xo_k_music_accuracy: 0.9353 - xo_k_brand_accuracy: 0.7869 - val_loss: 3.2497 - val_xo_polarity_loss: 0.5451 - val_xo_relevance_loss: 0.8878 - val_xo_k_spam_loss: 0.1418 - val_xo_k_joke_loss: 0.0279 - val_xo_k_political_loss: 0.2715 - val_xo_k_technology_loss: 0.4118 - val_xo_k_problem_loss: 0.1615 - val_xo_k_money_loss: 0.1010 - val_xo_k_music_loss: 0.2777 - val_xo_k_brand_loss: 0.4235 - val_xo_polari

13/13 [==============================] - 2s 150ms/step - loss: 2.7695 - xo_polarity_loss: 0.6417 - xo_relevance_loss: 0.3220 - xo_k_spam_loss: 0.1847 - xo_k_joke_loss: 0.1146 - xo_k_political_loss: 0.1718 - xo_k_technology_loss: 0.4214 - xo_k_problem_loss: 0.2114 - xo_k_money_loss: 0.1319 - xo_k_music_loss: 0.1498 - xo_k_brand_loss: 0.4203 - xo_polarity_accuracy: 0.7370 - xo_relevance_accuracy: 0.8697 - xo_k_spam_accuracy: 0.9470 - xo_k_joke_accuracy: 0.9734 - xo_k_political_accuracy: 0.9422 - xo_k_technology_accuracy: 0.7992 - xo_k_problem_accuracy: 0.9292 - xo_k_money_accuracy: 0.9686 - xo_k_music_accuracy: 0.9429 - xo_k_brand_accuracy: 0.7803 - val_loss: 3.2471 - val_xo_polarity_loss: 0.5461 - val_xo_relevance_loss: 0.8852 - val_xo_k_spam_loss: 0.1413 - val_xo_k_joke_loss: 0.0281 - val_xo_k_political_loss: 0.2723 - val_xo_k_technology_loss: 0.4111 - val_xo_k_problem_loss: 0.1616 - val_xo_k_money_loss: 0.1009 - val_xo_k_music_loss: 0.2762 - val_xo_k_brand_loss: 0.4242 - val_xo_polari

13/13 [==============================] - 2s 149ms/step - loss: 2.7771 - xo_polarity_loss: 0.6262 - xo_relevance_loss: 0.3228 - xo_k_spam_loss: 0.1997 - xo_k_joke_loss: 0.1171 - xo_k_political_loss: 0.1866 - xo_k_technology_loss: 0.4150 - xo_k_problem_loss: 0.1933 - xo_k_money_loss: 0.1570 - xo_k_music_loss: 0.1386 - xo_k_brand_loss: 0.4208 - xo_polarity_accuracy: 0.7454 - xo_relevance_accuracy: 0.8559 - xo_k_spam_accuracy: 0.9452 - xo_k_joke_accuracy: 0.9745 - xo_k_political_accuracy: 0.9322 - xo_k_technology_accuracy: 0.7989 - xo_k_problem_accuracy: 0.9348 - xo_k_money_accuracy: 0.9572 - xo_k_music_accuracy: 0.9426 - xo_k_brand_accuracy: 0.7811 - val_loss: 3.2418 - val_xo_polarity_loss: 0.5435 - val_xo_relevance_loss: 0.8829 - val_xo_k_spam_loss: 0.1414 - val_xo_k_joke_loss: 0.0278 - val_xo_k_political_loss: 0.2720 - val_xo_k_technology_loss: 0.4100 - val_xo_k_problem_loss: 0.1601 - val_xo_k_money_loss: 0.1011 - val_xo_k_music_loss: 0.2800 - val_xo_k_brand_loss: 0.4230 - val_xo_polari

13/13 [==============================] - 2s 151ms/step - loss: 2.8563 - xo_polarity_loss: 0.6648 - xo_relevance_loss: 0.3236 - xo_k_spam_loss: 0.1845 - xo_k_joke_loss: 0.1312 - xo_k_political_loss: 0.1908 - xo_k_technology_loss: 0.4253 - xo_k_problem_loss: 0.2195 - xo_k_money_loss: 0.1411 - xo_k_music_loss: 0.1539 - xo_k_brand_loss: 0.4215 - xo_polarity_accuracy: 0.7262 - xo_relevance_accuracy: 0.8604 - xo_k_spam_accuracy: 0.9449 - xo_k_joke_accuracy: 0.9721 - xo_k_political_accuracy: 0.9263 - xo_k_technology_accuracy: 0.7953 - xo_k_problem_accuracy: 0.9271 - xo_k_money_accuracy: 0.9625 - xo_k_music_accuracy: 0.9430 - xo_k_brand_accuracy: 0.7813 - val_loss: 3.2376 - val_xo_polarity_loss: 0.5433 - val_xo_relevance_loss: 0.8795 - val_xo_k_spam_loss: 0.1414 - val_xo_k_joke_loss: 0.0278 - val_xo_k_political_loss: 0.2746 - val_xo_k_technology_loss: 0.4103 - val_xo_k_problem_loss: 0.1610 - val_xo_k_money_loss: 0.1008 - val_xo_k_music_loss: 0.2771 - val_xo_k_brand_loss: 0.4218 - val_xo_polari

In [49]:
model.save("trained_on_all_large")

INFO:tensorflow:Assets written to: trained_on_all_large/assets


In [63]:
keras.utils.plot_model(model, to_file="model.png", show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
